# Non-Cartesian T1 | Subspace-Constrained Reconstruction
This tutorial uses the [`BART`](http://mrirecon.github.io/bart/) command-line interface (CLI) and presents how to work with radial data using BART, with the goal of implementing a full processing pipeline from raw data to T1 maps.


## Outline
1. Bart Setup
6. Reconstruction Pulseq Data
   1. Prepare data
   2. Subspace reconstruction
   3. Postprocessing
   


**Author**: [Vitali Telezki](mailto:vitali.telezki@med.uni-goettingen.de)

**Institution**: University Medical Center Göttingen

Based on previous material written by [Daniel Mackner](mailto:daniel.mackner@tugraz.at), [Moritz Blumenthal](mailto:blumenthal@@tugraz.at) and [Jon Tamir](mailto:jtamir@utexas.edu).

In this interactive session, we recommend running BART calls in the terminal, and use this notebook as a reference. Nevertheless, we will provide some examples of how to run BART commands in this notebook to be self-contained.

**💡CAVE:💡**

- **`%%bash` Cells in Jupyter Notebooks:**
  Magic commands like `%%bash` allow you to run bash commands directly within your notebook.
  Simply prefix your cell with `%%bash`, and you can run any shell command, including calls to BART tools, just as you would in a terminal.

- **`%%pybash` Cells in this Jupyter Notebook:**
   Since it is cumbersome to redefine bash variables over and over, we use a custom magic command `%%pybash` instead. This custom magic command allows to access python variables inside the `bash` environment, making execution more convenient and consistent. The caveat here is that variables need to be enclosed by `{}` and not preceeded by `$`. Be careful, when copy-pasting code to your terminal. We use `%%bash` wherever possible for simple translation.


## Python Requirements

In [ ]:
%%bash 
mkdir -p tutorial
cd tutorial 
git init
git remote add -f origin https://github.com/mrirecon/bart-workshop.git
git sparse-checkout init
git sparse-checkout set "dach_ismrm2025/subspace/src/"
git pull origin master
mv dach_ismrm2025/subspace/src/ ../

In [ ]:
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()


@register_cell_magic
def pybash(line, cell):
    ipython.run_cell_magic('bash', '', cell.format(**globals()))

## Bart Setup
### Google Colab
The cell will setup BART on Google Colab. For a detailed explanation, see **How to Run BART on Google Colaboratory** [here](https://github.com/mrirecon/bart-workshop/blob/master/ismrm2021/bart_on_colab/colab_gpu_tutorial.ipynb). You can skip this part if you want to run this notebook on your local machine.

This tutorial do need a GPU, you can select one by going to **Edit → Notebook settings**: Choose **GPU** from **Hardware accelerator** drop-down menu.

In [ ]:
%%bash

# Why Google?! As of May 13 cuda installed in colab (12.5) is incompatible with the driver (12.4)
# We install cuda 12.4 and link it here but it takes time!
apt install cuda-12-4

cd /usr/local
rm cuda
ln -s cuda-12.4 cuda

echo "GPU Information:"
nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv
nvcc --version


In [ ]:
%%bash
apt-get install -y make  gcc-12 g++-12 libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null


In [ ]:
%%bash
[ -d /content/bart ] && rm -r /content/bart

git clone https://github.com/mrirecon/bart/ bart &> /dev/null

Install BART

In [ ]:
%%bash
cd bart
COMPILE_SPECS=" PARALLEL=1
                CUDA=1
                CUDA_BASE=/usr/local/cuda
                CUDA_LIB=lib64
                OPENBLAS=1
                BLAS_THREADSAFE=1
                CC=gcc-12"


printf "%s\n" $COMPILE_SPECS > Makefiles/Makefile.local
make &> /dev/null


Set environment variables: enables correct path to BART

In [ ]:
# Define environment variables for BART and OpenMP
import os
import sys
os.environ['BART_TOOLBOX_PATH'] = "/content/bart"
os.environ['OMP_NUM_THREADS']="4"

# Add the BARTs toolbox to the PATH variable
os.environ['PATH'] = os.environ['BART_TOOLBOX_PATH'] + ":" + os.environ['PATH']
sys.path.append(os.environ['BART_TOOLBOX_PATH'] + "/python")

Test, if BART is installed correctly

In [ ]:
%%bash
echo "# BART used in this notebook:"
which bart
echo "# BART version: "
bart version

### Download phantom data. **Replace URL to get data we just recorded.**

In [ ]:
!wget https://????.????.???/download/ksp_irraga.ra -O meas.ra

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
# import os
# import sys
import src.plotSubspace as ps
# import numpy as np
# import importlib

mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use(['ggplot', 'src/customStyle.mplstyle'])

## Reconstruction Pulseq Data

Now, we aim to reconstruct data obtained with Pulseq and a sequence prepared by `bart seq`.
First, we create a signal dictionary, then we perform a SVD and finally create the temporal basis used in subspace reconstruction.


### Convert dat file to cfl
First, we need to convert Siemens `.dat` file to bart `.cfl/.hdr` file format using `bart twixread`. The file provided in this tutorial is already converted, simply call `bart copy` to generate `hdr` and `cfl` filesf from ist. Check raw data file dimensions.

In [ ]:
%%bash
bart copy meas.ra tmp_ksp
head -n2 tmp_ksp.hdr

We notice `PHS2_DIM` should be `TE_DM` instead.

In [ ]:
%%bash
bart transpose 2 5 tmp_ksp tmp_ksp
head -n2 tmp_ksp.hdr

### Trajectory and Sequence Params
You should have all relevant information from initial use of `bart seq`, when preparing Pulseq sequence. In case that is lost, we rerun the following commands.

In [ ]:
FIBO=377  # fibonacci number satisfying Nyquist when using RAGA
FRAMES=3
REP=int(FIBO*FRAMES)

In [ ]:
%%pybash

bart seq --IR_NON --raga -t {REP} -r {FIBO} grad mom adc
bart vec 1 1 0 tmp_a
bart extract 0 2 5 adc - | bart fmac - tmp_a traj
bart scale 0.5 traj traj
head -n2 traj.hdr

For subspace reco, all spokes in dim 5:

In [ ]:
%%pybash
bart reshape $(bart bitmask 2 10) 1 {REP} traj - | bart transpose 5 10 - traj
head -n2 traj.hdr

## Create Dictionary

The signal model according to Look-Locker in voxel $r$ at time $t$ is
$$
M(t,r)~=~M_{ss}(r)~-~\left[M_{ss}(r)~+~M_{0}(r)\right]~e^{-tR_1^*(r)}.~
$$

Using `bart signal` we create our signal dictionary with `nR1s` values for R1star and `nMss` values for steady-state magnetization.

In [ ]:
TR=0.00311 # [s] , BlockDurationRaster x DUR = 1e-05 x 311 = 0.00311
nR1s=1000
nMss=100

In [ ]:
%%pybash
### create vector with inversion times nREP*TR
bart index 5 {REP} - | bart scale {TR} - TI
bart signal -F -I -n {REP} -r {TR} -1 5e-3:5:{nR1s} -3 1e-2:1:{nMss} tmp_dict
# reshape the dictionary to have all the elements on the 6th dimension
bart reshape $(bart bitmask 6 7) $(({nR1s} * {nMss})) 1 tmp_dict tmp_dict1
head -n2 tmp_dict1.hdr
# squeeze the dicc1 before SVD
bart squeeze tmp_dict1 subspace_dict
head -n2 subspace_dict.hdr
# perform SVD on dictionary
bart svd -e subspace_dict U S V

### Visualize simulated dictionay and PCA coeffcients

In [ ]:
ps.plotSubspace()

### Estimate Coil Sensitivities Using Subspace
We realized, that 4 PCA coefficients contribute to more than 99% of the observed signal. Hence, we can reduce our basis to only contain the first 4 eigenvectors. We create a subspace basis spanned by those 4 eigenvectors.

In [ ]:
nCoe = 4 ## use 4 coefficients

In [ ]:
%%pybash

# create the temporal basis
bart extract 1 0 {nCoe} U subbasis
# transpose the basis to have time on the 6th dimension and coefficients on the 5th dimension
bart transpose 1 6 subbasis - | bart transpose 0 5 - subbasis

For consistency, check dimensions of subbasis, input data and trajectory:

In [ ]:
%%bash

head -n2 subbasis.hdr
head -n2 tmp_ksp.hdr
head -n2 traj.hdr



 Now, we estimate coil sensitivities using `bart ncalib` and reduced subspace basis in one go.

In [ ]:
%%bash

DEBUG_LEVEL=4 bart ncalib -g -N -i16 -t traj -B subbasis tmp_ksp sens_ncalib

### Perform reconstruction

Using estimated coil sensitvities, we can now perform parrallel imaging reconstruction with `bart pics`. We use the following parameter for iterations and regularization.

In [ ]:
ITER=100
REG=0.0005


In [ ]:
%%pybash
# # reconstruction with subspace constraint
bart pics -g -e -d5 -i {ITER} -R W:$(bart bitmask 0 1):$(bart bitmask 6):{REG} -B subbasis -t traj tmp_ksp sens_ncalib subspace_reco

We can visualize the first 4 coefficients projected onto our data set

In [ ]:
ps.plotCoefficientMaps()


In [ ]:
%%bash
## by chance, 2nd coefficient map also suitable for mask
bart slice 6 1 subspace_reco - | bart threshold -B 0.3 - mask
bart fmac mask subspace_reco subspace_reco

### Temporal Signal Evolution

We can also project our subbasis onto all spokes, resulting in temporal signal evolution.

In [ ]:
%%bash
bart fmac -s $(bart bitmask 6) subbasis subspace_reco imgs

In [ ]:
ps.plotTemporalEvolution()


### T1 Map from Reconstructed Images
Using `bart mobafit` we obtain T1 parameter map via pixel-wise fitting.

In [ ]:
%%pybash
### create vector with inversion times nREP*TR
bart index 5 {REP} - | bart scale {TR} - TI

DEBUG_LEVEL=4 bart mobafit -g -i20 --init 1:1:1 -B subbasis -L TI subspace_reco - | bart looklocker -t0 -D15.3e-3 - t1map

In [ ]:

ps.plotT1Map()